# A deep learning price prediction model with TensorFlow

# Requirements

Python libaries:
- TensorFlow
- keras
- pandas
- matplotlib
- seaborn

In [1]:
!conda create -n tf_env python tensorflow numpy pandas matplolib seaborn -y

Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - matplolib

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [2]:
!python -m ipykernel install --user --name tf_env --display-name "TF_ENV_1"

Installed kernelspec tf_env in /Users/ankitrawat/Library/Jupyter/kernels/tf_env


In [ ]:
# restart browser and check Kernel drop down box - change to TF_ENV

In [3]:
import tensorflow as tf
import pandas as pd

/Users/ankitrawat/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ankitrawat/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ankitrawat/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/ankitrawat/opt/anaconda3/lib/python3.7/site-pack

## Download the dataset

This section is optional.

When the given Intrinio API key expires, the above codes will fail to fetch from the server.
As a temporary workaround, load the dataset from disk.

In [5]:
df = pd.read_pickle('df_independent_2013_2018.pkl')
df.head(1)

,rsi,wr,vwap,adtv,ao,sma_5d,sma_15d,sma_30d,adj_close_price
date,,,,,,,,,
2018-12-28,35.88289,-62.894534,216.376505,4.693932e+07,-21.924007,153.422,161.806,171.163333,157.066371


In [6]:
df_close = pd.read_pickle('df_aapl_2013_2018.pkl')
df_close.head(1)

,adj_close_price
date,
2018-12-31,157.066371


## Splitting and scaling the data

In [ ]:
df_train = df['2017':'2013']
df_test = df[:'2018']

In [ ]:
df_test

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data = scaler.fit_transform(df_train.values)
test_data = scaler.transform(df_test.values)

In [ ]:
x_train = train_data[:, :-1]
y_train = train_data[:, -1]

In [ ]:
x_test = test_data[:, :-1]

## Building an artificial neural network with TensorFlow

### Phase 1: Assembling the graph

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
# import tensorflow as tf

num_features = x_train.shape[1]
num_features

In [ ]:
x = tf.placeholder(dtype=tf.float32, shape=[None, num_features])
y = tf.placeholder(dtype=tf.float32, shape=[None])

In [ ]:
nl_1, nl_2, nl_3, nl_4 = 512, 256, 128, 64

In [ ]:
# wi = tf.contrib.layers.variance_scaling_initializer(
#     mode='FAN_AVG', uniform=True, factor=1)

wi = tf.keras.initializers.VarianceScaling(scale=1.0, 
    mode='fan_avg', distribution='uniform')


zi = tf.zeros_initializer()

In [ ]:
# 4 Hidden layers
wt_hidden_1 = tf.Variable(wi([num_features, nl_1]))
bias_hidden_1 = tf.Variable(zi([nl_1]))

wt_hidden_2 = tf.Variable(wi([nl_1, nl_2]))
bias_hidden_2 = tf.Variable(zi([nl_2]))

wt_hidden_3 = tf.Variable(wi([nl_2, nl_3]))
bias_hidden_3 = tf.Variable(zi([nl_3]))

wt_hidden_4 = tf.Variable(wi([nl_3, nl_4]))
bias_hidden_4 = tf.Variable(zi([nl_4]))

# Output layer
wt_out = tf.Variable(wi([nl_4, 1]))
bias_out = tf.Variable(zi([1]))

In [ ]:
hidden_1 = tf.nn.relu(
    tf.add(tf.matmul(x, wt_hidden_1), bias_hidden_1))
hidden_2 = tf.nn.relu(
    tf.add(tf.matmul(hidden_1, wt_hidden_2), bias_hidden_2))
hidden_3 = tf.nn.relu(
    tf.add(tf.matmul(hidden_2, wt_hidden_3), bias_hidden_3))
hidden_4 = tf.nn.relu(
    tf.add(tf.matmul(hidden_3, wt_hidden_4), bias_hidden_4))
out = tf.transpose(tf.add(tf.matmul(hidden_4, wt_out), bias_out))

In [ ]:
mse = tf.reduce_mean(tf.squared_difference(out, y)) 

In [ ]:
optimizer = tf.train.AdamOptimizer().minimize(mse)

## Phase 2: training our model

In [ ]:
session = tf.InteractiveSession()

In [ ]:
session.run(tf.global_variables_initializer())

In [ ]:
from numpy import arange
from numpy.random import permutation

BATCH_SIZE = 100
EPOCHS = 100

for epoch in range(EPOCHS):
    # Shuffle the training data
    shuffle_data = permutation(arange(len(y_train)))
    x_train = x_train[shuffle_data]
    y_train = y_train[shuffle_data]

    # Mini-batch training
    for i in range(len(y_train)//BATCH_SIZE):
        start = i*BATCH_SIZE
        batch_x = x_train[start:start+BATCH_SIZE]
        batch_y = y_train[start:start+BATCH_SIZE]
        session.run(optimizer, feed_dict={x: batch_x, y: batch_y})

In [ ]:
[predicted_values] = session.run(out, feed_dict={x: x_test})

In [ ]:
predicted_scaled_data = test_data.copy()
predicted_scaled_data[:, -1] = predicted_values
predicted_values = scaler.inverse_transform(predicted_scaled_data)

In [ ]:
df_close[:'2018']

In [ ]:
predictions = predicted_values[:, -1][::-1]
actual = df_close[:'2018']['adj_close_price'].values[::-1]

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt

plt.figure(figsize=(12,8))
plt.title('Actual and predicted prices of AAPL 2018')
plt.plot(actual, label='Actual')
plt.plot(predictions, linestyle='dotted', label='Predicted')
plt.legend();